In [1]:
import numpy as np 
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

import shutil
import PIL
import os
import warnings
warnings.filterwarnings("ignore")

In [2]:
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.core import Activation,Dense, Dropout
from keras.layers import MaxPool2D, Conv2D, Flatten, BatchNormalization
from keras.utils import np_utils
from keras.applications import VGG19, ResNet50, VGG16, Xception
import keras

import tensorflow as tf

Using TensorFlow backend.


In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test_vc2kHdQ.csv")

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
test.shape

In [ ]:
# image = Image.open("images/1960.jpg")
# TEST YOUR VERSION OF PILLOW
# Run this cell. If you see a picture of a cat you're all set!
with Image.open('images/1960.jpg') as im:
    display(im)

In [ ]:
data.shape

In [ ]:
path = "./images/"

img_names = []

for folder, subfolder, filenames in os.walk(path):
    for img in filenames:
        img_names.append(folder+'\\'+img)
        
print('Images: ',len(img_names))

In [ ]:
# Creating a datafram

img_sizes = []
rejected =[]

for item in img_names:
    try:
        with Image.open(item) as img:
            img_sizes.append(img.size)
            
    except:
        rejected.append(item)
        
        
print(f"Image Size is: {len(img_sizes)}")
print(f"Rejects : {len(rejected)}")

In [ ]:
df = pd.DataFrame(img_sizes)
df[0].describe()

In [ ]:
image1 = Image.open("./images/1960.jpg")
print(image1.size)
display(image1)

In [ ]:
r, g, b = image1.getpixel((0, 0))
print(r,g,b)

In [ ]:
transform = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(p = 0.8),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])

im = transform(image1)

print(im.shape)
plt.imshow(np.transpose(im.numpy(), (1,2,0)))

In [ ]:
# Lets create train and test transforms


train_transform = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(p = 0.8),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
])


test_transform = transforms.Compose([
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
])




In [ ]:
train['emergency_or_not'].value_counts()

In [ ]:
train.head()

In [ ]:
train['image_names'][0]

In [ ]:
# Lets create new folders 
os.makedirs('./data/train/0')
os.makedirs('./data/train/1')
os.makedirs('./data/val/0')
os.makedirs('./data/val/1')
os.makedirs('./data/test/')

In [ ]:
index_p = int(len(train)*0.7)
df_train = train[:index_p]
df_val = train[index_p:]

In [ ]:
df_train.reset_index(inplace= True, drop = True)
df_val.reset_index(inplace = True, drop = True)

In [ ]:
df_val.head()

In [ ]:
# lets copy images to train folders

for i in range(len(df_train)):
    if(df_train['emergency_or_not'][i] == 0):
        shutil.copy('./images/{}'.format(df_train['image_names'][i]), './data/train/0')
    else:
        shutil.copy('./images/{}'.format(df_train['image_names'][i]), './data/train/1')
        

In [ ]:
# lets copy images to val folders

for i in range(len(df_val)):
    if(df_val['emergency_or_not'][i] == 0):
        shutil.copy('./images/{}'.format(df_val['image_names'][i]), './data/val/0')
    else:
        shutil.copy('./images/{}'.format(df_val['image_names'][i]), './data/val/1')
        

In [ ]:
df_val['emergency_or_not'].value_counts()

In [ ]:
# lets copy images to test folders

for i in range(len(test)):
    shutil.copy('./images/{}'.format(test['image_names'][i]), './data/test')
        

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)


In [5]:
train_dir = "./data1/train"
val_dir = "./data1/val"

train_generator = train_datagen.flow_from_directory(train_dir, target_size= (224,224), batch_size = 16, class_mode= "binary")
val_generator = val_datagen.flow_from_directory(val_dir, target_size = (224,224), batch_size = 16, class_mode= "binary")

Found 18936 images belonging to 2 classes.
Found 8059 images belonging to 2 classes.


In [24]:
18936/16

1183.5

In [25]:
8059/16

503.6875

In [ ]:
for b, labels_batch in train_generator:
    print("data_batch_shape: ", b.shape)
    print("labels batch_shape: ", labels_batch.shape)
    break

In [ ]:
from fastai.vision import *

In [26]:
conv_base = Xception(weights = "imagenet",
                 include_top = False, 
                 input_shape = (224,224,3))

In [27]:
conv_base.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [28]:
conv_base.trainable = True

set_trainable = False

for layer in conv_base.layers:
    
    if(layer.name == 'block5_conv4'):
        set_trainable= True
    if (set_trainable):
        layer.trainable = True
    else:
        layer.trainable = False

In [29]:
conv_base.trainable = False

In [30]:
conv_base.summary()

Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
___________________________________________________________________________________________

In [31]:
# conv_base.trainable = False

In [32]:

model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.6))
# model.add(Dense(1024))
# model.add(Activation("relu"))
# model.add(Dropout(0.6))
model.add(Dense(1))
model.add(Activation("sigmoid"))


opt = keras.optimizers.Adam(learning_rate=3e-5, beta_1= 0.9, beta_2=0.999)
model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy'])

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Model)             (None, 7, 7, 2048)        20861480  
_________________________________________________________________
flatten_4 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_7 (Dense)              (None, 512)               51380736  
_________________________________________________________________
batch_normalization_12 (Batc (None, 512)               2048      
_________________________________________________________________
activation_56 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                

In [33]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU')]

In [35]:
from keras.callbacks import EarlyStopping



history = model.fit_generator(train_generator, steps_per_epoch = 1184, epochs = 50, validation_data = val_generator,
                             validation_steps = 504)

Epoch 1/50
1184/1184 [==============================] - 174s 147ms/step - loss: 0.4652 - accuracy: 0.7973 - val_loss: 0.8546 - val_accuracy: 0.7588
Epoch 2/50
1184/1184 [==============================] - 176s 149ms/step - loss: 0.3511 - accuracy: 0.8499 - val_loss: 0.8817 - val_accuracy: 0.8290
Epoch 3/50
1184/1184 [==============================] - 177s 149ms/step - loss: 0.3099 - accuracy: 0.8697 - val_loss: 0.4164 - val_accuracy: 0.8312
Epoch 4/50
1184/1184 [==============================] - 176s 149ms/step - loss: 0.2790 - accuracy: 0.8836 - val_loss: 0.0302 - val_accuracy: 0.8283
Epoch 5/50
1184/1184 [==============================] - 175s 148ms/step - loss: 0.2582 - accuracy: 0.8915 - val_loss: 0.6986 - val_accuracy: 0.8005
Epoch 6/50
1184/1184 [==============================] - 176s 148ms/step - loss: 0.2428 - accuracy: 0.8992 - val_loss: 0.7922 - val_accuracy: 0.8082
Epoch 7/50
1184/1184 [==============================] - 176s 149ms/step - loss: 0.2242 - accuracy: 0.9077 - val_

KeyboardInterrupt: 

In [ ]:
model.save_weights('model_wieghts_layer6.h5')
model.save('model_keras_layer6.h5')

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator

# train_generator = ImageDataGenerator(rescale= 1/255., 
#                                     rotation_range= 40,
#                                     width_shift_range= 0.2,
#                                     height_shift_range=0.2,
#                                     shear_range = 0.2,
#                                     zoom_range= 0.2,
#                                     horizontal_flip= True)

# test_generator = ImageDataGenerator(rescale = 1/255.)

# train_gen = train_generator.flow(xtrain, ytrain, batch_size=32)
# test_gen = test_generator.flow(xtest, ytest, batch_size=32)


In [ ]:
# model.fit_generator(train_gen,
# #                     validation_data=test_gen, epochs= 100, verbose=1
#                    )

In [ ]:
from PIL import Image

img = []
count = 0
for i in range(test.shape[0]):
    image = Image.open(f"images/{test['image_names'][i]}")
    image = np.array(image)
    image = image.astype('float32')
    image /= 255.0
    img.append(image)

In [ ]:
plt.imshow(img[0])

In [ ]:
test.shape

In [ ]:
len(img)

In [ ]:
img[0].shape

In [ ]:
for image, label in (train_generator):
        break

In [ ]:
im = make_grid(image, nrow=5)
# Print the images
plt.figure(figsize=(12,4))
plt.imshow(np.transpose(image))

In [ ]:
predictions = []

for image in img:
    
    pred = model.predict_classes(image.reshape(1,224,224,3))
    predictions.append(pred)

In [ ]:
predictions

In [ ]:
np.array(predictions).reshape(706,)

In [ ]:
sam = pd.read_csv("sample_submission_yxjOnvz.csv")

In [ ]:
sam['emergency_or_not'] = np.array(predictions).reshape(706,)

In [ ]:
sam.head()

In [ ]:
sam.to_csv("Sample_keras3.csv", index = False)